In [ ]:
import pandas as pd 

# concept.json


In [ ]:
## 1. Dịch dữ liệu

In [ ]:
```python
import pandas as pd
import os
import json
from googletrans import Translator

# Initialize translator
translator = Translator()

# Function to translate text
def translate_text(text):
    try:
        translated = translator.translate(text, dest='en')
        return translated.text
    except Exception as e:
        print(f"Error translating text: {e}")
        return text

# Load progress if there is any
progress_file = 'concept_progress.json'
if os.path.exists(progress_file):
    with open(progress_file, 'r', encoding='utf-8') as f:
        translated_data = [json.loads(line) for line in f]
else:
    translated_data = []

# Read the input file line by line
input_file = 'concept_part_1.json'
total_lines = sum(1 for line in open(input_file, 'r', encoding='utf-8'))
processed_lines = len(translated_data)

with open(input_file, 'r', encoding='utf-8') as f, open(progress_file, 'a', encoding='utf-8') as pf:
    for i, line in enumerate(f):
        if i < processed_lines:
            continue  # Skip already processed lines
        data = json.loads(line)
        
        # Convert to DataFrame
        df = pd.DataFrame([data])
    
        
        # Translate the remaining columns
        for column in df.columns:
            if df[column].dtype == object:
                df[column] = df[column].apply(lambda x: [translate_text(item) for item in x] if isinstance(x, list) else translate_text(x))
        
        # Convert back to dictionary
        translated_data.append(df.to_dict(orient='records')[0])
        
        # Save the translated line to progress file
        pf.write(json.dumps(translated_data[-1], ensure_ascii=False) + '\n')
        
        # Output progress
        if (i + 1) % 10 == 0 or (i + 1) == total_lines:
            print(f"Processed {i + 1}/{total_lines} lines")
```

In [ ]:
df = pd.read_json('concept_translated.json', lines=True)
df.head()

In [ ]:
## 2. Khám phá dữ liệu

In [ ]:
### 2.1. Thống kê mô tả

In [ ]:
# Thống kê cơ bản
df.describe()

In [ ]:
## 3. Làm sạch dữ liệu

In [ ]:
### 3.1. Xử lý dữ liệu thiếu

In [ ]:
print("Số lượng giá trị thiếu trong từng cột:")
print(df.isnull().sum())

In [ ]:
### 3.2 Xử lý dữ liệu trùng lặp

In [ ]:
print("Số lượng bản ghi trùng lặp:", df.duplicated().sum())
df = df.drop_duplicates()
print("Số lượng trùng lặp sau khi xử lý:", df.duplicated().sum())

# course-field.json

In [ ]:
# Đọc dữ liệu

In [ ]:
import pandas as pd
import json
import re

from googletrans import Translator

In [ ]:
df = pd.read_json('course-field.json', lines=True)
df.head()

In [ ]:
# 1. Dịch dữ liệu

In [ ]:
```python
import pandas as pd
from googletrans import Translator

# Khởi tạo Translator
translator = Translator()

# Đọc dữ liệu từ file JSON
df = pd.read_json('course-field.json', lines=True)

# Lọc lấy các cột cần thiết
df_to_translate = df[['course_id', 'course_name', 'field']]

# Hàm để dịch từng chuỗi
def translate_text(text, src_lang='zh-cn', dest_lang='en'):
    try:
        # Thực hiện dịch ngôn ngữ
        translation = translator.translate(text, src=src_lang, dest=dest_lang)
        return translation.text
    except Exception as e:
        print(f"Lỗi dịch: {e}")
        return text  # Trả về chuỗi gốc nếu gặp lỗi
```

In [ ]:
```python
# Dịch cột 'course_name'
df_to_translate['course_name'] = df_to_translate['course_name'].apply(lambda x: translate_text(x))
# Dịch từng mục trong cột 'field' (là một danh sách)
df_to_translate['field'] = df_to_translate['field'].apply(lambda fields: [translate_text(field) for field in fields])
# Ghi kết quả đã dịch ra file JSON mới với định dạng mỗi đối tượng trên một dòng
with open('course-field-translated_2.json', 'w', encoding='utf-8') as f:
    for _, row in df_to_translate.iterrows():
        # Chuyển đổi từng dòng thành định dạng JSON
        json.dump(row.to_dict(), f, ensure_ascii=False)
        f.write('\n')  # Xuống dòng sau mỗi đối tượng JSON
```

In [ ]:
df = pd.read_json('course-field-translated.json', lines=True)
df.head()

In [ ]:
# 2. Khám phá dữ liệu

In [ ]:
## 2.1. Thống kê mô tả

In [ ]:
# 1. Số lượng khóa học
num_courses = df['course_id'].nunique()
print(f"Số lượng khóa học: {num_courses}")

# 2. Số lượng các lĩnh vực khác nhau
unique_fields = set(field for fields_list in df['field'] for field in fields_list)
num_unique_fields = len(unique_fields)
print(f"Số lượng các lĩnh vực khác nhau: {num_unique_fields}")

In [ ]:
# 3. Phân bố số lượng khóa học theo từng lĩnh vực
field_distribution = df.explode('field')['field'].value_counts()
print("\nPhân bố số lượng khóa học theo từng lĩnh vực:")
print(field_distribution)



In [ ]:
# 4. Phân bố độ dài tên khóa học (số ký tự)
course_name_length = df['course_name'].apply(len)
print("\nThống kê độ dài tên khóa học:")
print(course_name_length.describe())

In [ ]:
## 2.2. Trực quan hóa dữ liệu

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Explode the 'field' column to get individual field entries and count occurrences
field_distribution = df.explode('field')['field'].value_counts()

# Get the top 10 fields and calculate the sum of "Others"
top_10_fields = field_distribution.nlargest(10)
others_count = field_distribution[10:].sum()

# Prepare data for the pie chart
labels = top_10_fields.index.tolist() + ['Others']
sizes = top_10_fields.values.tolist() + [others_count]

# Create the circle (pie) chart
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('viridis', len(labels)))
plt.title('Phân bố số lượng khóa học theo từng lĩnh vực (Top 10 + Others)', fontsize=16)
plt.show()

In [ ]:
# 3. Làm sạch dữ liệu

In [ ]:
## 3.1. Xử lý dữ liệu thiếu

In [ ]:
# Kiểm tra dữ liệu thiếu
print("Số lượng giá trị thiếu trong từng cột:")
print(df.isnull().sum())

# Tỷ lệ dữ liệu thiếu
missing_ratio = df.isnull().mean() * 100
print("\nTỷ lệ dữ liệu thiếu trong từng cột (%):")
print(missing_ratio)

# Xử lý dữ liệu thiếu
# 1. Loại bỏ các hàng chứa giá trị thiếu
df_dropped = df.dropna()

# Kiểm tra lại số lượng giá trị thiếu sau khi xử lý
print("\nSố lượng giá trị thiếu sau khi xử lý:")
print(df_dropped.isnull().sum())  # Nếu dùng phương pháp loại bỏ
# print(df.isnull().sum())  # Nếu dùng phương pháp thay thế

Số lượng giá trị thiếu trong từng cột:
course_id      0
course_name    0
field          0
dtype: int64

Tỷ lệ dữ liệu thiếu trong từng cột (%):
course_id      0.0
course_name    0.0
field          0.0
dtype: float64

Số lượng giá trị thiếu sau khi xử lý:
course_id      0
course_name    0
field          0
dtype: int64


In [ ]:
## 3.2. Xử lý dữ liệu nhiễu

In [ ]:
# Hàm loại bỏ ký tự không mong muốn
def clean_text(text):
    # Loại bỏ ký tự đặc biệt
    text = re.sub(r'[^\w\s]', '', text)  # Giữ lại chữ cái, số và khoảng trắng
    text = text.strip()  # Loại bỏ khoảng trắng thừa ở hai đầu
    text = text.lower()  # Chuyển về viết thường
    return text


# Tiến hành xử lý cho cột 'course_name'
df['cleaned_course_name'] = df['course_name'].apply(clean_text)

# Kiểm tra dữ liệu đã xử lý
df[['course_id', 'course_name', 'cleaned_course_name']].head()


In [ ]:
## 3.3. Xử lý dữ liệu trùng lặp

In [ ]:
# Chuyển đổi cột 'field' thành chuỗi
df['field'] = df['field'].apply(lambda x: ', '.join(x))

# Kiểm tra dữ liệu trùng lặp
duplicate_rows = df.duplicated()

# Đếm số lượng bản ghi trùng lặp
num_duplicates = duplicate_rows.sum()
print(f"Số lượng bản ghi trùng lặp: {num_duplicates}")

# Hiển thị các bản ghi trùng lặp
if num_duplicates > 0:
    print("Các bản ghi trùng lặp:")
    print(df[duplicate_rows])

# Xóa bản ghi trùng lặp (nếu cần)
df_cleaned = df.drop_duplicates()

# Kiểm tra lại số lượng bản ghi sau khi xóa trùng lặp
print(f"Số lượng bản ghi sau khi xóa trùng lặp: {len(df_cleaned)}")